In [0]:
import pyspark

IMORT_PYSPARK FUNCTION

In [0]:
from pyspark import * 
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
# training.ashish.productdescription_bronzetraining.ashish.productcategory_silver# from pyspark.sql.functions import monotonically_increasing_id,current_timestamp

LOAD DATA FROM SOURCE LOC TO TARGET LOC USING COPY COMMAND 

In [0]:
source_location = 'abfss://trainingexternal@bayadapoc.dfs.core.windows.net/learners/ashish/clientlocation/AdventureWorks_Ashish/'
target_location = 'abfss://trainingexternal@bayadapoc.dfs.core.windows.net/learners/ashish/landing/AdventureWorks_Ashish/'

dbutils.fs.cp(source_location, target_location, recurse = True)

CREATE TABLE CONTROL FOR INGEST WHAT DATA TO INGEST,FROM AND WHERE AND TO WHERE

In [0]:
%sql
CREATE TABLE IF NOT EXISTS training.ashish.data_movement_control (
  datamovement_id INT,
  source_system STRING,
  source_details STRING,
  target_details STRING,
  active BOOLEAN,
  target_data_zone STRING,
  created_date TIMESTAMP,
  created_by STRING,
  modified_date TIMESTAMP,
  modified_by STRING
)

CREATE LOGGING TABLE FOR TRACING WHICH FILES WERE INGESTED WHEN AND WITH WHAT STATUS

In [0]:
# %sql
# CREATE TABLE IF NOT EXISTS training.ashish.data_movement_logging (
#   log_id STRING,
#   source_system STRING,
#   table_id STRING,
#   source_details STRING,
#   target_details STRING,
#   event_start_time TIMESTAMP,
#   event_end_time TIMESTAMP,
#   status STRING,
#   watermark_value TIMESTAMP
# )

In [0]:
%sql
-- drop table training.ashish.data_movement_logging

LOAD DATA MANUALLY IN CONTROL TABLE USING INSERT INTO COMMAND 

In [0]:
# %sql
# INSERT  INTO training.ashish.data_movement_control VALUES (
#   13
#   , 'AdventureWork_ashish'
#   , 'abfss://trainingexternal@bayadapoc.dfs.core.windows.net/learners/ashish/clientlocation/AdventureWorks_Ashish/ProductSubcategory/'
#   , 'abfss://trainingexternal@bayadapoc.dfs.core.windows.net/learners/ashish/landing/AdventureWorks_ashish/ProductSubcategory/'
#   ,true
#   , 'landing'
#   , current_timestamp()
#   , current_user()
#   , null
#   , null
# )

UPDATE QUERY IF WE WANT UPDATE IN TABLE 

In [0]:
# %sql
# update training.ashish.data_movement_control
# set target_details = "abfss://trainingexternal@bayadapoc.dfs.core.windows.net/learners/ashish/landing/AdventureWorks_ashish/ProductDescription/"
# where datamovement_id = 8

In [0]:
%sql

select * from training.ashish.data_movement_control

CREATE A FUNCTION  AUTOMATES THE INGESTION OF NEW OR UPDATED FILE BASED ON METADATA RULES ENUSE NO DUPLICATES PROCESSING USING WATERMARK LOGIC AND MAINTAING A DETAILED TRIAL OF EVERY FILE MOVEMENT.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import to_unix_timestamp
from datetime import datetime
import uuid
def ingest_data_to_landing_reference():
    control_df= spark.sql('select * from training.ashish.data_movement_control where active')
    log_df= spark.sql('select * from training.ashish.data_movement_logging')
    for row in control_df.collect():
        id = row[0]
        source_files= dbutils.fs.ls(row[2])
        # print(source_files)
        watermark_value = log_df.filter(log_df['table_id'] == id).agg(max("watermark_value").alias('latest_watermark_value')).collect()[0]['latest_watermark_value']
        # print(watermark_value)
        date = watermark_value
        print(date)
        if date == None:
            date = datetime(1900,1,1)
        else:
            pass
        date= int(date.timestamp() * 1000)
        # print(date)
        for files in source_files:
            # print("files:", files)
            filepath, modification_time = files.path, files.modificationTime
            # print("modification_time:", modification_time)
            print("filepath:",filepath)
            if modification_time > date:
                # print('Inside the loop')
                dbutils.fs.cp(filepath, row[3] + '/' + files.name, recurse = True)
                table_id = row[0]
                source_details = row[2]  + files.name
                # print("src",source_details)
                target_details = row[3] + files.name
                source_system = row[1]
                log_id = str(uuid.uuid4())
                Status = 'SUCCESS'
                temp_query = spark.sql(f"""INSERT INTO training.ashish.data_movement_logging select
                        '{log_id}'
                          , '{source_system}'
                          , '{table_id}'
                          , '{source_details}'
                          , '{target_details}'
                          , current_timestamp()
                          , current_timestamp()
                          , '{Status}'
                          , current_timestamp()
                """)
    return 'Success'

CALL THE FUNCTION

In [0]:
ingest_data_to_landing_reference()

In [0]:
%sql
select * from training.ashish.data_movement_logging
-- select * from training.ashish.data_movement_control limit 5;

---truncate table training.ashish.data_movement_logging


In [0]:
%sql
-- alter table training.ashish.data_movement_control
-- drop column watermark_column

In [0]:
%sql
select * from training.ashish.dim_address

In [0]:
%sql
-- update training.ashish.data_movement_control
-- set target_details = "abfss://trainingexternal@bayadapoc.dfs.core.windows.net/learners/ashish/landing/AdventureWorks_ashish/BusinessEntityAddress/"
-- where datamovement_id = 3

In [0]:
%sql
-- TRUNCATE TABLE training.ashish.data_movement_logging

In [0]:
%sql
-- select * from training.ashish.data_movement_logging
--truncate table training.ashish.data_movement_logging;

In [0]:
%sql
-- UPDATE training.ashish.data_movement_control
-- SET active = true 
-- where datamovement_id = 11

In [0]:
%sql

-- drop table training.ashish.dim_address
drop table training.ashish.dim_product
-- drop table training.ashish.dim_date
---drop table training.ashish.fact_salesorderdetail

In [0]:
%python
# 1. Reading all tables
product = spark.sql("select * from training.ashish.product_silver")
product_model = spark.sql("select * from training.ashish.productmodel_silver")
product_subcategory = spark.sql("select * from training.ashish.productsubcategory_silver")
product_category = spark.sql("select * from training.ashish.productcategory_silver")
product_description = spark.sql("select * from training.ashish.productdescription_silver")
productmodelproductdescription = spark.sql("select * from training.ashish.productmodelproductdescriptionculture_silver")
culture = spark.sql("select * from training.ashish.culture_silver")

# display(product)
# display(product_model)
# display(product_subcategory)
# display(product_category)
# display(product_description)
# display(productmodelproductdescription)
# display(culture)

# 2. Joining all tables
df_product = product_subcategory \
    .join(product_category, product_subcategory["ProductCategoryID"] == product_category["ProductCategoryID"], "inner")
              #  .join(product_category, product_subcategory["ProductCategoryID"] == product_category["ProductCategoryID"], "inner") \
              #  .join(productmodelproductdescription, product_model["ProductModelID"] == productmodelproductdescription["ProductModelID"], "inner")\
              #  .join(product_description, productmodelproductdescription["ProductDescriptionID"] == product_description["ProductDescriptionID"], "inner") \
              #  .join(culture, productmodelproductdescription["CultureID"] == culture["CultureID"], "inner")
display(df_product)


# # 2. Select final columns
# product_df = df_product.select(
#                product["ProductID"].alias("ProductID"),
#                product["Name"].alias("Product_Name"),
#                product["ProductNumber"],
#                product['MakeFlag'],
#                product["FinishedGoodsFlag"],
#                product["Color"],
#                product['StandardCost'],
#                product_model['ProductModelID'].alias('ProductModelID'),
#                product_model['Name'].alias('ProductModelName'),
#                product_subcategory["ProductSubcategoryID"].alias("ProductSubcategoryID"),
#                product_subcategory["Name"].alias("ProductSubcategoryName"),
#                product_category['ProductCategoryID'].alias("ProductCategoryID"),
#                product_category["Name"].alias("ProductCategoryName"),
#                product["ModifiedDate"],
#                product_description["ProductDescriptionID"].alias("ProductDescription"),
#                product_description["Description"].alias("Description"),
#                culture["CultureID"].alias("CultureID"),
#                culture["Name"].alias("CultureName")
#                ) 
# display(product_df)

In [0]:
%sql
SELECT * FROM training.ashish.dim_product;


In [0]:
%sql
select * from training.ashish.address_bronze

In [0]:
%sql
select * from training.ashish.emailaddress_bronze

In [0]:
%sql
select count(ProductSubcategoryID) from training.ashish.productsubcategory_silver

In [0]:
%sql
select count(ProductCategoryID) from training.ashish.productcategory_silver

In [0]:
%sql
select count(ProductDescriptionID) from training.ashish.productdescription_silver --762

In [0]:
%sql
select count(ProductDescriptionID) from training.ashish.productmodelproductdescriptionculture_silver